In [ ]:
!pip install transformers

In [7]:

import pandas as pd
import numpy as np

from tqdm import tqdm
from textwrap import wrap


# Select and import model
model_name = "sberbank-ai/rugpt3large_based_on_gpt2" #@param ["sberbank-ai/mGPT", "sberbank-ai/rugpt3small_based_on_gpt2", "sberbank-ai/rugpt3medium_based_on_gpt2", "sberbank-ai/rugpt3large_based_on_gpt2", "bigscience/bloom-1b1", "bigscience/bloom-1b7", "bigscience/bloom-3b1", "facebook/xglm-564M", "facebook/xglm-1.7B"]

from transformers import GPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.cuda()

In [1]:
!nvidia-smi

Wed Dec 14 13:52:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.126.02   Driver Version: 418.126.02   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:8A:00.0 Off |                    0 |
| N/A   36C    P0    44W / 300W |     11MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
cd ~

/data/home/asurikov


In [3]:
!ls

ASPE-NEWS  ASTE  README.ipynb  cherniak  tmp


In [ ]:
model_name = "ASTE/rugpt3small_based_on_gpt2_finetuned-50x3"
model_name = "ASTE/rugpt3large_based_on_gpt2_finetuned-50x3"
model_name = "ASTE/rugpt3l"
from transformers import GPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.cuda()

# Prepare data

In [9]:
data_path_car = "ASTE/SRE-15/SentiRuEval_car_train_short_edited.csv"
data_path_rest = "ASTE/SRE-15/SentiRuEval_rest_train_short_edited.csv"
data_path_film = "ASTE/SRE-15/SentiRuEval_film_train_short_edited.csv"

In [10]:
data_car = pd.read_csv(data_path_car)
data_rest = pd.read_csv(data_path_rest)
data_film = pd.read_csv(data_path_film)

In [11]:
def print_linesplit(text, chars):
  print('\n'.join(wrap(text, chars)))

def format_columns(text, response):
  return 'Текст: ' + text.strip() + '\n' + response.strip()

def format_input(text):
  return 'Текст: ' + text.strip() + '\n'

def get_aspects(text):
  text = text.strip()
  responses = text.split('\n')
  return list(set([response.split(': ')[0].strip().lower() for response in responses]))

def atomize_triplet(aspect, opinions, polarity):
  opinions = opinions.split(', ')
  return [[aspect, opinion, polarity] for opinion in opinions] 

def get_triplets(text):
  text = text.strip()
  responses = text.split('\n')
  group = []
  for response in responses:
    spans = [span.strip().lower() for span in response.split(': ')]
    if len(spans) != 3:
      print("Error-triplet:")
      print(spans)
    else:
      group.extend(atomize_triplet(*spans))
  return group

texts_car =  list(map(format_input, data_car["Text"]))
texts_rest = list(map(format_input, data_rest["Text"]))
texts_film = list(map(format_input, data_film["Text"]))
triplets_car =  list(map(get_triplets, data_car["Response"]))
triplets_rest = list(map(get_triplets, data_rest["Response"]))
triplets_film = list(map(get_triplets, data_film["Response"]))
aspects_car =  list(map(get_aspects, data_car["Response"]))
aspects_rest = list(map(get_aspects, data_rest["Response"]))
aspects_film = list(map(get_aspects, data_film["Response"]))

print(texts_car[0])
print("Аспекты: ", aspects_car[0])
print("Триплеты: ", triplets_car[0])
print("\n")
print(texts_rest[0])
print("Аспекты: ", aspects_rest[0])
print("Триплеты: ", triplets_rest[0])
print("\n")
print(texts_film[0])
print("Аспекты: ", aspects_film[0])
print("Триплеты: ", triplets_film[0])

Текст: Недавно купил этот автомобиль. Авто отличное! Двигатель 2,5 литра, турбодизель. Прежний хозяин сказал при продаже, что масло не жрет, солярку тоже, летит как угорелая! Так оно и есть. 140 км/ч нормальная крейсерская скорость. Вообще немцы умеют делать автомобили. Дорогу держит отлично, так как достаточно широкая машина. Тормоза все дисковые. Главное передний привод, по сравнению с другими немецкими автомобилями. Такими как мерседес и бмв этого же класса. Места в автомобиле очень много. Не тесно даже, когда сидят пять взрослых человек. Багажное отделение тоже огромно. Влезла стиральная машина. По соотношению цена - качество, отличный автомобиль. Больше никогда не сяду за руль российского автомбиля! Всем рекомендую Ауди.

Аспекты:  ['двигатель', 'багажное отделение', 'места', 'авто', 'тормоза', 'соотношение цена - качество', 'дорогу держит', 'передний привод', 'прежний хозяин', 'крейсерская скорость']
Триплеты:  [['авто', 'отличное', 'положительно'], ['двигатель', '2,5 литра', 'по

In [12]:
model_name_short = model_name.split('/')[1]
model_name_short 

car_ATE_res_path = 'ASTE/SRE-15/evals/car_short_train_ATE_' + model_name_short + '.txt'
rest_ATE_res_path = 'ASTE/SRE-15/evals/rest_short_train_ATE_' + model_name_short + '.txt'
film_ATE_res_path = 'ASTE/SRE-15/evals/film_short_train_ATE_' + model_name_short + '.txt'

car_ASTE_res_path = 'ASTE/SRE-15/evals/car_short_train_ASTE_' + model_name_short + '.txt'
rest_ASTE_res_path = 'ASTE/SRE-15/evals/rest_short_train_ASTE_' + model_name_short + '.txt'
film_ASTE_res_path = 'ASTE/SRE-15/evals/film_short_train_ASTE_' + model_name_short + '.txt'

# n-shot Prompts (rest)

In [ ]:
p6shot_rest_ATE = "\n\n".join([
    "Текст: Обслуживанием вполне довольны, официанты были приветливы, хотя время ожидания оказалось дольше обещанного. \nОбъекты: обслуживание, официанты, время ожидания",
    "Текст: Еще из недостатков хочется отметить ужасный запах в туалете и не самые чистые скатерти >=(. \nОбъекты: запах в туалете, скатерти",
    "Текст: Столиков постоянно не хватает. Многих позиций в меню не было в наличии, что огорчило. Но подача блюд была отличной! \nОбъекты: столики, позиции меню, подача блюд",
    "Текст: Администратор зала нахамил нам и не стал разбираться с нашей проблемой, сказал что раньше надо было записываться. Отвратительно! Спасибо официанту Виталию, он был приветлив и ему единственному было не наплевать на нас. \nОбъекты: администратор зала, официант Виталий",
    "Текст: Стейки, ризотто и пудинги просто бомба! Интерьер не самый потрясающий, но мы обожаем это заведение и всем рекомендуем хотя бы раз сюда попасть. \nОбъекты: Стейки, ризотто, пудинги, интерьер",
    "Текст: Пирог был сухим и пересоленным. Цены в целом нормальные. Ожидал большего... \nОбъекты: пирог, цены",
])

p6shot_rest_ASOE = "\n\n".join([
    "Текст: Обслуживанием вполне довольны, официанты были приветливы, хотя время ожидания оказалось дольше обещанного. \nобслуживание: довольны \nофицианты: приветливы \nвремя ожидания: дольше обещанного",
    "Текст: Еще из недостатков хочется отметить ужасный запах в туалете и не самые чистые скатерти >=(. \nзапах в туалете: ужасный \nскатерти: не самые чистые",
    "Текст: Столиков постоянно не хватает. Многих позиций в меню не было в наличии, что огорчило. Но подача блюд была отличной! \nстолики: не хватает \nпозиции меню: не было в наличии \nподача блюд: отличная",
    "Текст: Администратор зала нахамил нам и не стал разбираться с нашей проблемой, сказал что раньше надо было записываться. Отвратительно! Спасибо официанту Виталию, он был приветлив и ему единственному было не наплевать на нас. \nадминистратор зала: нахамил, не стал разбираться \nофициант Виталий: приветлив, не наплевать",
    "Текст: Стейки, ризотто и пудинги просто бомба! Интерьер не самый потрясающий, но мы обожаем это заведение и всем рекомендуем хотя бы раз сюда попасть. \ncтейки: просто бомба \nризотто: просто бомба \nпудинги: просто бомба \nинтерьер: не самый потрясающий",
    "Текст: Пирог был сухим и пересоленным. Цены в целом нормальные. Ожидал большего... \nпирог: сухой, пересоленный \nцены: нормальные",
])

p6shot_rest_ASTE = "\n\n".join([
    "Текст: Обслуживанием вполне довольны, официанты были приветливы, хотя время ожидания оказалось дольше обещанного. \nобслуживание: довольны: положительно \nофицианты: приветливы: положительно \nвремя ожидания: дольше обещанного: негативно",
    "Текст: Еще из недостатков хочется отметить ужасный запах в туалете и не самые чистые скатерти >=(. \nзапах в туалете: ужасный: негативно \nскатерти: не самые чистые: негативно",
    "Текст: Столиков постоянно не хватает. Многих позиций в меню не было в наличии, что огорчило. Но подача блюд была отличной! \nстолики: не хватает: негативно \nпозиции меню: не было в наличии: негативно \nподача блюд: отличная: положительно",
    "Текст: Администратор зала нахамил нам и не стал разбираться с нашей проблемой, сказал что раньше надо было записываться. Отвратительно! Спасибо официанту Виталию, он был приветлив и ему единственному было не наплевать на нас. \nадминистратор зала: нахамил, не стал разбираться: негативно \nофициант Виталий: приветлив, не наплевать: положительно",
    "Текст: Стейки, ризотто и пудинги просто бомба! Интерьер не самый потрясающий, но мы обожаем это заведение и всем рекомендуем хотя бы раз сюда попасть. \ncтейки: просто бомба: положительно \nризотто: просто бомба: положительно \nпудинги: просто бомба: положительно \nинтерьер: не самый потрясающий: нейтрально",
    "Текст: Пирог был сухим и пересоленным. Цены в целом нормальные. Ожидал большего... \nпирог: сухой, пересоленный: негативно \nцены: нормальные: нейтрально",
])

# n-shot Prompts (car)

In [ ]:
p6shot_car_ATE = "\n\n".join([
    "Текст: Салон выполнен на совесть, швы окуратные дизайн сдержанный но далеко не простенький, качественная кожа и отличная шумоизоляция. \nОбъекты: салон, швы, дизайн, кожа, шумоизоляция",
    "Текст: Еще из недостатков хочется отметить слабое лакокрасочное покрытие, все открывается только с ключа. \nОбъекты: лакокрасочное покрытие",
    "Текст: Отсутствует русский язык в бортовом компьютере и CD-проигрывателе. и много еще проблем. \nОбъекты: бортовой компьютер, CD-проигрыватель",
    "Текст: Ходовка не смотря на то что крепкая, но очень сильно подвержена коррозии и в случае если вам нужно будет что то заменить, то открутить будет тяжело. \nОбъекты: ходовка",
    "Текст: Кстати есть слабое место — провода идущие к задним амортизаторам бывает обламываются в месте входа в корпус и подвеска становится в жесткий режим и не регулируется. \nОбъекты: провода, подвеска",
    "Текст: В общем машина надежная и неприхотливая, дефекты легко ремонтируются! \nОбъекты: машина, дефекты",
])

p6shot_car_ASOE = "\n\n".join([
    "Текст: Салон выполнен на совесть, швы аккуратные, дизайн сдержанный но далеко не простенький, качественная кожа и отличная шумоизоляция. \nсалон: на совесть \nшвы: аккуратные \nдизайн: сдержанный \nкожа: качественная \nшумоизоляция: отличная",
    "Текст: Еще из недостатков хочется отметить слабое лакокрасочное покрытие, все открывается только с ключа. \nлакокрасочное покрытие: слабое",
    "Текст: Отсутствует русский язык в бортовом компьютере и CD-проигрывателе. и много еще проблем. \nбортовой компьютер: отсутствует русский язык \nCD-проигрыватель: отсутствует русский язык",
    "Текст: Ходовка не смотря на то что крепкая, но очень сильно подвержена коррозии и в случае если вам нужно будет что то заменить, то открутить будет тяжело. \nходовка: крепкая, сильно подвержена коррозии",
    "Текст: Кстати есть слабое место — провода идущие к задним амортизаторам бывает обламываются в месте входа в корпус и подвеска становится в жесткий режим и не регулируется. \nпровода: обламываются \nподвеска: становится в жесткий режим, не регулируется",
    "Текст: В общем машина надежная и неприхотливая, дефекты легко ремонтируются! \nмашина: надежная, неприхотливая \nдефекты: легко ремонтируются",
])

p6shot_car_ASTE = "\n\n".join([
    "Текст: Салон выполнен на совесть, швы аккуратные, дизайн сдержанный но далеко не простенький, качественная кожа и отличная шумоизоляция. \nсалон: на совесть: положительно \nшвы: аккуратные: положительно \nдизайн: сдержанный: нейтрально \nкожа: качественная: положительно \nшумоизоляция: отличная: положительно",
    "Текст: Еще из недостатков хочется отметить слабое лакокрасочное покрытие, все открывается только с ключа. \nлакокрасочное покрытие: слабое: негативно",
    "Текст: Отсутствует русский язык в бортовом компьютере и CD-проигрывателе. и много еще проблем. \nбортовой компьютер: отсутствует русский язык: негативно \nCD-проигрыватель: отсутствует русский язык: негативно",
    "Текст: Ходовка не смотря на то что крепкая, но очень сильно подвержена коррозии и в случае если вам нужно будет что то заменить, то открутить будет тяжело. \nходовка: крепкая: положительно \nходовка:  сильно подвержена коррозии: негативно",
    "Текст: Кстати есть слабое место — провода идущие к задним амортизаторам бывает обламываются в месте входа в корпус и подвеска становится в жесткий режим и не регулируется. \nпровода: обламываются: негативно \nподвеска: становится в жесткий режим: негативно \nподвеска: не регулируется: негативно",
    "Текст: В общем машина надежная и неприхотливая, дефекты легко ремонтируются! \nмашина: надежная, неприхотливая: положительно \nдефекты: легко ремонтируются: положительно",
])


p12shot_car_ATE = "\n\n".join([
    "Текст: Салон выполнен на совесть, швы окуратные дизайн сдержанный но далеко не простенький, качественная кожа и отличная шумоизоляция. \nОбъекты: салон, швы, дизайн, кожа, шумоизоляция",
    "Текст: Еще из недостатков хочется отметить слабое лакокрасочное покрытие, все открывается только с ключа. \nОбъекты: лакокрасочное покрытие",
    "Текст: Отсутствует русский язык в бортовом компьютере и CD-проигрывателе. и много еще проблем. \nОбъекты: бортовой компьютер, CD-проигрыватель",
    "Текст: Ходовка не смотря на то что крепкая, но очень сильно подвержена коррозии и в случае если вам нужно будет что то заменить, то открутить будет тяжело. \nОбъекты: ходовка",
    "Текст: Кстати есть слабое место — провода идущие к задним амортизаторам бывает обламываются в месте входа в корпус и подвеска становится в жесткий режим и не регулируется. \nОбъекты: провода, подвеска",
    "Текст: В общем машина надежная и неприхотливая, дефекты легко ремонтируются! \nОбъекты: машина, дефекты",
    "Текст: Трескается кожа на руле и боковинах комбинированных сидений. \nОбъекты: кожа",
    "Текст: Одна из характерных \"болячек\" модели - текущие трубки охлаждения масла. \nОбъекты: трубки охлаждения масла",
    "Текст: Из-за грязи и влаги перестает работать мотор привода порогов.\nОбъекты: мотор привода порогов",
    "Текст: Такие показатели долговечности по праву можно считать едва ли не выдающимися. \nОбъекты: показатели долговечности",
    "Текст: Довольно часто отваливается провод и держатель обогрева заднего стекла. \nОбъекты: провод, держатель",
])

p12shot_car_ASOE = "\n\n".join([
    "Текст: Очень удобные кресла, можно с легкостью отправляться в дальние путешествия и не бояться за усталость. \nкресла: удобные",
    "Текст: Салон выполнен на совесть, швы аккуратные, дизайн сдержанный но далеко не простенький, качественная кожа и отличная шумоизоляция. \nсалон: на совесть \nшвы: аккуратные \nдизайн: сдержанный \nкожа: качественная \nшумоизоляция: отличная",
    "Текст: Еще из недостатков хочется отметить слабое лакокрасочное покрытие, все открывается только с ключа. \nлакокрасочное покрытие: слабое",
    "Текст: Отсутствует русский язык в бортовом компьютере и CD-проигрывателе. и много еще проблем. \nбортовой компьютер: отсутствует русский язык \nCD-проигрыватель: отсутствует русский язык",
    "Текст: Ходовка не смотря на то что крепкая, но очень сильно подвержена коррозии и в случае если вам нужно будет что то заменить, то открутить будет тяжело. \nходовка: крепкая, сильно подвержена коррозии",
    "Текст: Кстати есть слабое место — провода идущие к задним амортизаторам бывает обламываются в месте входа в корпус и подвеска становится в жесткий режим и не регулируется. \nпровода: обламываются \nподвеска: становится в жесткий режим, не регулируется",
    "Текст: В общем машина надежная и неприхотливая, дефекты легко ремонтируются! \nмашина: надежная, неприхотливая \nдефекты: легко ремонтируются",
    "Текст: Трескается кожа на руле и боковинах комбинированных сидений. \nкожа на руле: трескается \nкожа на боковинах сидений: трескается",
    "Текст: Одна из характерных \"болячек\" модели - текущие трубки охлаждения масла. \nтрубки охлаждения масла: текущие",
    "Текст: Из-за грязи и влаги перестает работать мотор привода порогов. \nмотор привода порогов: перестает работать",
    "Текст: Такие показатели долговечности по праву можно считать едва ли не выдающимися. \nпоказатели долговечности: выдающиеся",
    "Текст: Довольно часто отваливается провод и держатель обогрева заднего стекла. \nпровод: часто отваливается \nдержатель: часто отваливается",
])

p12shot_car_ASTE = "\n\n".join([
    "Текст: Очень удобные кресла, можно с легкостью отправляться в дальние путешествия и не бояться за усталость. \nкресла: удобные: положительно",
    "Текст: Салон выполнен на совесть, швы аккуратные, дизайн сдержанный но далеко не простенький, качественная кожа и отличная шумоизоляция. \nсалон: на совесть: положительно \nшвы: аккуратные: положительно \nдизайн: сдержанный: нейтрально \nкожа: качественная: положительно \nшумоизоляция: отличная: положительно",
    "Текст: Еще из недостатков хочется отметить слабое лакокрасочное покрытие, все открывается только с ключа. \nлакокрасочное покрытие: слабое: негативно",
    "Текст: Отсутствует русский язык в бортовом компьютере и CD-проигрывателе. и много еще проблем. \nбортовой компьютер: отсутствует русский язык: негативно \nCD-проигрыватель: отсутствует русский язык: негативно",
    "Текст: Ходовка не смотря на то что крепкая, но очень сильно подвержена коррозии и в случае если вам нужно будет что то заменить, то открутить будет тяжело. \nходовка: крепкая: положительно \nходовка:  сильно подвержена коррозии: негативно",
    "Текст: Кстати есть слабое место — провода идущие к задним амортизаторам бывает обламываются в месте входа в корпус и подвеска становится в жесткий режим и не регулируется. \nпровода: обламываются: негативно \nподвеска: становится в жесткий режим: негативно \nподвеска: не регулируется: негативно",
    "Текст: В общем машина надежная и неприхотливая, дефекты легко ремонтируются! \nмашина: надежная, неприхотливая: положительно \nдефекты: легко ремонтируются: положительно",
    "Текст: Трескается кожа на руле и боковинах комбинированных сидений. \nкожа на руле: трескается: негативно \nкожа на боковинах сидений: трескается: негативно",
    "Текст: Одна из характерных \"болячек\" модели - текущие трубки охлаждения масла. \nтрубки охлаждения масла: текущие: негативно",
    "Текст: Из-за грязи и влаги перестает работать мотор привода порогов. \nмотор привода порогов: перестает работать: негативно",
    "Текст: Такие показатели долговечности по праву можно считать едва ли не выдающимися. \nпоказатели долговечности: выдающиеся: положительно",
    "Текст: Довольно часто отваливается провод и держатель обогрева заднего стекла. \nпровод: часто отваливается: негативно \nдержатель: часто отваливается: негативно",
])

# Test model - ATE

In [ ]:
index = 83
prompt = "".join([
    p12shot_car_ATE,
    "\n\n",
    texts_car[index],
    "Объекты:"
])

input_ids = tokenizer.encode(prompt, return_tensors="pt").cuda()
out = model.generate(
        input_ids, 
        min_length=100, 
        max_length=800, 
        eos_token_id=5, 
        pad_token_id=1,
        top_k=15,
        top_p=0.0,
        no_repeat_ngram_size=5
)
generated_text = list(map(tokenizer.decode, out))[0]

print("")
print_linesplit(texts_car[index], 150)
print("\nAspects found by model:")
print(generated_text[len(prompt):].strip().split('\n')[0])
print("\nAspects from gold annotation:")
print(aspects_car[index])

# Test model - ASTE with specified aspect term

In [ ]:
index = 14
aspect = "комплектация"
prompt = "".join([
    p6shot_car_ASTE,
    "\n\n",
    texts_car[index],
    aspect + ":"
])

input_ids = tokenizer.encode(prompt, return_tensors="pt").cuda()
out = model.generate(
        input_ids, 
        min_length=100, 
        max_length=800, 
        eos_token_id=5, 
        pad_token_id=1,
        top_k=10,
        top_p=0.0,
        no_repeat_ngram_size=4
)
generated_text = list(map(tokenizer.decode, out))[0]


print("")
print_linesplit(texts_car[index], 150)
print(f"\nOpinions for aspect {aspect} found by model:")
print(generated_text[len(prompt):].strip().split('\n')[0])
print("\nTriplets from gold annotation:")
print(triplets_car[index])

# Test model - ASTE (Joint)

In [13]:
from textwrap import wrap

prompt = texts_car[55]
input_ids = tokenizer.encode(prompt, return_tensors="pt")
input_ids = input_ids.cuda()
out = model.generate(
        input_ids, 
        min_length=100, 
        max_length=400, 
        eos_token_id=5, 
        pad_token_id=1,
        top_k=1,
        top_p=0.0,
        no_repeat_ngram_size=5
)
generated_text = list(map(tokenizer.decode, out))[0]
#print('\n' + '\n'.join(wrap(prompt, 100)) + '\n')
print(generated_text[len(prompt):].split("\n\n")[0])
#print(generated_text[len(prompt):].split("\n"))

машина: радует хорошим комфортом: положительно 
салон: хороший: положительно 
подвеска: мягкая: положительно 
дорогу держит уверенно: положительно 
просторный и комфортый салон: положительно 
мягкая подвеска: с одной стороны хорошо: положительно 
наших дорогах машина идет в раскочку: негативно</s>


# Run dataset eval on model - ATE

Run car dataset

In [ ]:
car_results = []

for index in tqdm(range(len(texts_car))):
  prompt = "".join([
      p6shot_car_ATE,
      "\n\n",
      texts_car[index],
      "Объекты:"
  ])

  input_ids = tokenizer.encode(prompt, return_tensors="pt").cuda()
  out = model.generate(
          input_ids, 
          min_length=100, 
          max_length=800, 
          eos_token_id=5, 
          pad_token_id=1,
          top_k=15,
          top_p=0.0,
          no_repeat_ngram_size=5
  )
  generated_text = list(map(tokenizer.decode, out))[0]

  car_results.append(generated_text[len(prompt):].strip().split('\n')[0])

In [ ]:
with open(car_ATE_res_path, "w") as file:
    file.write("\n".join(car_results))

Run rest dataset

In [ ]:
rest_results = []

for index in tqdm(range(len(texts_rest))):
  prompt = "".join([
      p6shot_rest_ATE,
      "\n\n",
      texts_rest[index],
      "Объекты:"
  ])

  input_ids = tokenizer.encode(prompt, return_tensors="pt").cuda()
  out = model.generate(
          input_ids, 
          min_length=100, 
          max_length=800, 
          eos_token_id=5, 
          pad_token_id=1,
          top_k=15,
          top_p=0.0,
          no_repeat_ngram_size=5
  )
  generated_text = list(map(tokenizer.decode, out))[0]

  rest_results.append(generated_text[len(prompt):].strip().split('\n')[0])

In [ ]:
with open(rest_res_path, "w") as file:
    file.write("\n".join(rest_results))

# Run dataset eval on model - ASTE (Joint)

In [14]:
def run_dataset_eval_aste_joint(texts):
    results = []
    for index in tqdm(range(len(texts))):
        prompt = texts[index]

        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        input_ids = input_ids.cuda()
        out = model.generate(
                input_ids, 
                min_length=100, 
                max_length=500, 
                eos_token_id=5, 
                pad_token_id=1,
                top_k=5,
                top_p=0.0,
                no_repeat_ngram_size=5
        )
        generated_text = list(map(tokenizer.decode, out))[0]

        results.append(generated_text[len(prompt):].strip().split("\n\n")[0])
    return results

Run car dataset

In [15]:
texts_car_lim = texts_car[50:]
car_results = run_dataset_eval_aste_joint(texts_car_lim)

100%|██████████| 45/45 [11:38<00:00, 15.52s/it]


In [16]:
with open(car_ASTE_res_path, "w") as file:
    file.write("\n\n".join([group for group in car_results]))

In [17]:
car_results[0:5]

['Машина: понравилась: положительно \nмашина: не ломалась, не смотря на возраст: положительно \nсалон: удобный, комфортный: положительно \nместа: в салоне много: положительно \nзапчасти: подходят от других автомобилей: положительно \nкузов: гниют задние арки: негативно \nцена: относительно низкая: положительно</s>',
 'салон: большой: положительно \nбагажник: большой: положительно\nуправление: удобное: положительно \nпередачи: переключаются плавно: положительно \nкушает: много: негативно \nцена: выгодная: положительно</s>',
 'машина: предназначена для фанатов этой серий машин: положительно \nкомплектация: базовая: нейтрально \nбаза: расширенная: положительно \nкузов: не всё так гладко: негативно \nстеклоподьемники: нет: негативно \nрезина: неубиваемая: положительно \nаккумулятор: не заряжен: негативно \nстартер: не крутит: негативно \nсвечи: не горят: негативно \nдвигатель: не дымит: негативно \nкузов не гнилой: положительно \nстартер не крутит: положительно \nсветотехника: не работает:

Run rest dataset

In [18]:
texts_rest_lim = texts_rest[50:]
rest_results = run_dataset_eval_aste_joint(texts_rest_lim)

100%|██████████| 44/44 [10:17<00:00, 14.04s/it]


In [19]:
with open(rest_ASTE_res_path, "w") as file:
    file.write("\n\n".join([group for group in rest_results]))

In [20]:
rest_results[0:5]

['первое впечатление: ужасное: негативно \nсалат: ужаснейший: негативно \nкурица: сухая: негативно \nсоус: невкусный: негативно \nдесерт: понравились: положительно \nпирожные: вкусные: положительно \nторт: из шоколада: положительно \nвпечатление: остались голодными: негативно \nеда: понравилась: положительно \nеда: остались голодными, недовольны: негативно</s>',
 'кухня: отличная: положительно \nинтерьер: выше всяких похвал: положительно \nместа: довольно много: положительно \nсалат цезарь: вкусный: положительно \nроллы: вкусные: положительно \nсуши: вкусные: положительно\nхлеб: вкусный: положительно\nсоус: странный: негативно \nмисо суп: не очень, одна вода: негативно \nтирамису: понравился: положительно \nтирамиссу: долго ждали: негативно \nофициант: не очень вежливый: негативно \nвремя ожидания: долго: негативно \nцену: не очень, много: негативно \nсоус: не очень, вкус странный: негативно</s>',
 'обслуживание: порадовало: положительно \nадминистратор Екатерина: встретила нас и посад

Run film dataset

In [21]:
texts_film_lim = texts_film[50:]
film_results = run_dataset_eval_aste_joint(texts_film_lim)

100%|██████████| 43/43 [08:57<00:00, 12.51s/it]


In [22]:
with open(film_ASTE_res_path, "w") as file:
    file.write("\n\n".join([group for group in film_results]))

In [23]:
film_results[0:5]

['фильм: редко балует нас такими историями: негативно \nистория: потрясающе теплая: положительно \nцвета: мягкие коричнево—золотые тона напоминают горячий шоколад: положительно \nфильм: оставляет ощущение покоя и уютности: положительно \nпотрясающе теплый: положительно \nпослевкусие: очень приятное: положительно</s>',
 'фильм: не таким, каким ожидал: негативно \nпаутина: на протяжении всего фильма: раздражает, непонятки: негативно \nсобытия: последовательный сюжет: нейтрально \nпоказания приборов: нереальные: негативно \nплан: продуманный: положительно \nсценарий: полнейший: положительно \nконцовка фильма: потрясающая: положительно \nзал: аплодировал: положительно \nзакадровый голос: заставил задуматься: положительно \nфильм: заслуживает «десятки»: положительно \nконца фильма: потрясающей: положительно</s>',
 'Русалочка: мультфильм: потрясающий во всех смыслах: положительно \nмультфильм: поистине потрясающий во всем отношении: положительно \nБэль: полностью соответствует данному ей име

# Methods for metrics

In [35]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer, util
st_model = SentenceTransformer('inkoziev/sbert_synonymy')

Defaulting to user installation because normal site-packages is not writeable


Some weights of the model checkpoint at /data/home/asurikov/.cache/torch/sentence_transformers/inkoziev_sbert_synonymy/ were not used when initializing BertModel: ['embeddings.position_ids']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
#This method returns precision, recall, F1 from TP, FP, FN
def prf1(TP, FP, FN):
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1 = 2 * precision * recall / (precision + recall)
    return precision, recall, F1

#This method finds longest common sequence
def LCS(a, b):
    if len(a) == 0 or len(b) == 0:
        return 0
    res = max(LCS(a[:-1], b), LCS(a, b[:-1]))
    if a[-1] == b[-1]:
        res = max(res, LCS(a[:-1], b[:-1]) + 1)
    return res

#This method takes two lists of strings and returnsut ids of left difference, intersection and right difference
#and a dictionary of ids in the right list that correspond to id from the left
#SBERT (st_model) and Transformer (tokenizer) tokenizers should be constructed before use
def get_span_ps_sets_ids(left, right, p_ratio = 0.75, s_ratio = 0.5):
    #exit if one list is empty
    if not left:
      return [], [], list(range(len(right))), {}
    
    if not right:
      return list(range(len(left))), [], [], {}

    #Compute embeddings for both lists

    embs_left = st_model.encode(left, convert_to_tensor=True)
    embs_right = st_model.encode(right, convert_to_tensor=True)
    tf_ids_left = [tokenizer.encode(span, return_tensors="pt").tolist()[0] for span in left]
    tf_ids_right = [tokenizer.encode(span, return_tensors="pt").tolist()[0] for span in right]

    left_indices = set()
    right_indices = set()
    pairs = {}
    
    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embs_left, embs_right)

    for n in range(cosine_scores.size(0)):
      for m in range(cosine_scores.size(1)):

        #Compute longest common sequence
        lcs_len = LCS(tf_ids_left[n], tf_ids_right[m])
        lcs_ratio_left = lcs_len / len(tf_ids_left[n])
        lcs_ratio_right = lcs_len / len(tf_ids_right[m])
        
        #We find pairs of strings to be equal if LCS takes up more than [s_ratio] in one of tokenized
        #representations, or if SBERT is more than [r_ratio] confident in their similarity
        if (cosine_scores[n][m] >= p_ratio or lcs_ratio_left >= s_ratio or lcs_ratio_right >= s_ratio):
          left_indices.add(n)
          right_indices.add(m)
          if n not in pairs.keys():
            pairs[n] = [m]
          else:
            pairs[n].append(m)

    res_left = list(set(range(len(left))).difference(left_indices))
    res_right = list(set(range(len(right))).difference(right_indices))

    return res_left, list(left_indices), res_right, pairs

#Basic check - works for all kids of tasks
def get_metrics_strict(pred, target):
  TP, FP, FN = 0, 0, 0
  for i in range(len(pred)):
    TP += len([x for x in pred[i] if x in target[i]])
    FP += len([x for x in pred[i] if x not in target[i]])
    FN += len([x for x in target[i] if x not in pred[i]])
  
  return TP, FP, FN

def convert_polarity(span):
  if span == 'отрицательно' or span == 'негативно':
    return -1
  if span == 'нейтрально':
    return 0
  if span == 'положительно' or span == 'позитивно':
    return 1
  else:
    #error-code
    return -2

#paraphrases & subsequenses
def get_metrics_ATE_ps(pred, target, p_ratio = 0.75, s_ratio = 0.5):
  TP, FP, FN = 0, 0, 0
  for i in range(len(pred)):
    left, intersection, right, pairs = get_span_ps_sets_ids(pred[i], target[i], p_ratio, s_ratio)
    TP += len(intersection)
    FP += len(left)
    FN += len(right)
  return TP, FP, FN

#paraphrases & subsequenses
def get_metrics_ASTE_ps(pred, target, p_ratio = 0.75, s_ratio = 0.5):
  #Aspect
  TP_A, FP_A, FN_A = 0, 0, 0
  #Pair
  TP_P, FP_P, FN_P = 0, 0, 0
  #Triplet
  TP_T, FP_T, FN_T = 0, 0, 0

  groups_A = []
  groups_O = []

  #All the groups of triplets = the amount of exapmles in dataset
  for group_i in range(len(pred)):
    
    #We find aspect/opinion sets venn diagram
    left_A, intersection_A, right_A, pairs_A = get_span_ps_sets_ids([triplet[0] for triplet in pred[group_i]],
                                                                    [triplet[0] for triplet in target[group_i]], p_ratio, s_ratio)
    left_O, intersection_O, right_O, pairs_O = get_span_ps_sets_ids([triplet[1] for triplet in pred[group_i]],
                                                                    [triplet[1] for triplet in target[group_i]], p_ratio, s_ratio)
    


    groups_A.append([{
        pred[group_i][key][0] : [target[group_i][value][0] for value in values]
        } for key, values in pairs_A.items()])
    groups_O.append([{
        pred[group_i][key][1] : [target[group_i][value][1] for value in values]
        } for key, values in pairs_O.items()])

    #metrics for ATE
    TP_A += len(intersection_A)
    FP_A += len(left_A)
    FN_A += len(right_A)
    
    pair_found_pred_ids = set()
    pair_found_target_ids = set()
    triplet_found_pred_ids = set()
    triplet_found_target_ids = set()

    for pred_triplet_i in intersection_A:
      for target_triplet_i in pairs_A[pred_triplet_i]:
        #if both A and O span pairs are found to be similar - this is a vaild pair
        if pred_triplet_i in pairs_O.keys() and target_triplet_i in pairs_O[pred_triplet_i]:
          pair_found_pred_ids.add(pred_triplet_i)
          pair_found_target_ids.add(target_triplet_i)

          #convert to polarity classes
          pred_pc = convert_polarity(pred[group_i][pred_triplet_i][2])
          target_pc = convert_polarity(target[group_i][target_triplet_i][2])

          #if both polarity classes are equal AND not error-class - this is a valid triplet
          if pred_pc == target_pc and pred_pc != -2:
            triplet_found_pred_ids.add(pred_triplet_i)
            triplet_found_target_ids.add(target_triplet_i)
    
    TP_P += len(pair_found_pred_ids)
    FP_P += len(set(range(len( pred[group_i] ))).difference(pair_found_pred_ids))
    FN_P += len(set(range(len( target[group_i] ))).difference(pair_found_target_ids))

    TP_T += len(triplet_found_pred_ids)
    FP_T += len(set(range(len( pred[group_i] ))).difference(triplet_found_pred_ids))
    FN_T += len(set(range(len( target[group_i] ))).difference(triplet_found_target_ids))
  
  return [TP_A, FP_A, FN_A], [TP_P, FP_P, FN_P], [TP_T, FP_T, FN_T], groups_A, groups_O

In [28]:
#loading in results
def load_ATE_results(path):
  results = ""
  with open(path, "r") as file:
    results = file.read()
  return [line.strip().split(', ') for line in results]

def load_ASTE_results(path):
  results = ""
  with open(path, "r") as file:
    results = file.read()
  
  #wrong_format
  wf = 0
  #right_format
  rf = 0
  #wrong_polarity - not in any class
  wp = 0
  #right_polarity
  rp = 0
  #wrong_polarity_format - polarity contains junk
  wpf = 0
  #right_polarity_format
  rpf = 0

  res_triplets = []

  for group in results.split('\n\n'):
    group_triplets = []
    for triplet in group.split('\n'):
      spans = [span.strip() for span in triplet.split(': ')]
      if len(spans) != 3:
        wf += 1
      else:
        rf += 1
        
        #we check polarity span - does it contain junk?
        polarity_span = spans[2].split()
        if (len(polarity_span) > 1):
          wpf += 1
          spans[2] = polarity_span[0]
        else:
          rpf += 1
        
        #we check polarity class
        polarity_converted = convert_polarity(spans[2])
        if polarity_converted == -2:
          wp += 1
        else:
          rp += 1

        group_triplets.extend(atomize_triplet(*spans))
    res_triplets.append(group_triplets)

  f_rate, p_rate, pf_rate = 0, 0, 0

  if wf + rf > 0:
    f_rate = wf / (wf + rf)
  if wp + rp > 0:
    p_rate = wp / (wp + rp)
  if wpf + rpf > 0:
    pf_rate = wpf / (wpf + rpf)
  
  return res_triplets, f_rate, p_rate, pf_rate

# Get metrics - ATE

In [ ]:
results_rest = load_ATE_results(rest_ATE_res_path)
results_car = load_ATE_results(car_ATE_res_path)

In [ ]:
df_res = pd.DataFrame(columns=('precision', 'recall', 'F1'))

df_res.loc[1] = prf1(*get_metrics_strict(results_rest, aspects_rest))
df_res.loc[2] = prf1(*get_metrics_strict(results_car, aspects_car))
df_res.loc[3] = prf1(*get_metrics_ATE_ps(results_rest, aspects_rest))
df_res.loc[4] = prf1(*get_metrics_ATE_ps(results_car, aspects_car))

df_res.index = ['rest_ATE_strict', 'car_ATE_strict', 'metrics_rest_ATE_ps', 'metrics_car_ATE_ps']

df_res

# Get metrics - ASTE

In [56]:
res_triplets_rest, f_rate_rest, p_rate_rest, pf_rate_rest = load_ASTE_results(rest_ASTE_res_path)
res_triplets_car, f_rate_car, p_rate_car, pf_rate_car = load_ASTE_results(car_ASTE_res_path)

print(f"Model: {model_name_short}")

df_load = pd.DataFrame(columns=('format error', 'polarity error-class', 'polarity format error'))
df_load.loc[1] = [f_rate_rest, p_rate_rest, pf_rate_rest]
df_load.loc[2] = [f_rate_car, p_rate_car, pf_rate_car]

df_load.index = ['SRE-15_short_rest',
                'SRE-15_short_car']
df_load

Model: rugpt3small_based_on_gpt2_finetuned-50+50


,format error,polarity error-class,polarity format error
SRE-15_short_rest,0.086667,0.003650,0.003650
SRE-15_short_car,0.122093,0.003311,0.006623


In [57]:
#FIX GPT-3 RESULTS - ONLY USE WITH LEGACY GPT3 RESULT FILE WITH LEN(RES) = 96
#res_triplets_rest_ = res_triplets_rest[:8] + res_triplets_rest[9:46] + res_triplets_rest[47:]

In [58]:
triplets_rest = triplets_rest[50:]
triplets_car = triplets_car[50:]

In [59]:
print(len(res_triplets_rest), len(triplets_rest))
print(len(res_triplets_car), len(triplets_car))

44 44
45 45


In [60]:
res_triplets_rest[25]

[['обслуживание', 'ни в какие рамки', 'негативно'],
 ['интерьер', 'очень темный', 'негативно'],
 ['интерьер', 'не всякому будет по вкусу', 'негативно']]

In [61]:
df_res = pd.DataFrame(columns=('precision', 'recall', 'F1'))

metrics_A_rest, metrics_P_rest, metrics_T_rest, groups_A_rest, groups_O_rest = get_metrics_ASTE_ps(res_triplets_rest, triplets_rest, p_ratio = 0.85, s_ratio = 0.6)
metrics_A_car, metrics_P_car, metrics_T_car, groups_A_car, groups_O_car = get_metrics_ASTE_ps(res_triplets_car, triplets_car, p_ratio = 0.85, s_ratio = 0.6)

df_res.loc[1] = prf1(*metrics_A_rest)
df_res.loc[2] = prf1(*metrics_P_rest)
df_res.loc[3] = prf1(*metrics_T_rest)
df_res.loc[4] = prf1(*get_metrics_strict(res_triplets_rest, triplets_rest))
df_res.loc[5] = prf1(*metrics_A_car)
df_res.loc[6] = prf1(*metrics_P_car)
df_res.loc[7] = prf1(*metrics_T_car)
df_res.loc[8] = prf1(*get_metrics_strict(res_triplets_car, triplets_car))

df_res.index = ['rest_Aspect_ps',
                'rest_Pair_ps',
                'rest_Triplet_ps',
                'rest_Triplet_strict',
                'car_Aspect_ps',
                'car_Pair_ps',
                'car_Triplet_ps',
                'car_Triplet_strict',]#, 'metrics_rest_ASTE_ps', 'metrics_car_ASTE_ps']

df_res

TypeError: forward() got an unexpected keyword argument 'return_dict'

In [ ]:
print(f"Model: {model_name_short}")
df_res

In [ ]:
groups_O_car

In [ ]:
def dict_string_rep(pair_dict):
  groups = []
  for group in pair_dict:
    groups.append(
        "\n".join(
            [f"{key}: {', '.join(values)}"for key, values in group]
        )
    )

In [ ]:
dict_string_rep(groups_O_car)